## 0. Setup

In [201]:
#standard packages
import pandas as pd
import numpy as np
#import warnings
import warnings
warnings.filterwarnings('ignore')
import os
import collections

#time packages
import datetime as dt


In [202]:
#from setup import main as setup
#setup(env='sagemaker', project='cc_cid_calculator')
os.chdir('/Users/luiz.superti/Documents/GitHub/alice-coding/CausalInference-Analysis/cc_cid_calculator/')
print(os.getcwd())  # Prints the current working directory

/Users/luiz.superti/Documents/GitHub/alice-coding/CausalInference-Analysis/cc_cid_calculator


***

## 1. extract

In [203]:
df = pd.read_csv('src/data/cid_care_cost_did_calc_df.csv')
df.columns

Index(['member_internal_code', 'calendar_date', 'mth_order', 'total_n_mth',
       'current_sex', 'current_gender', 'member_age', 'current_city',
       'current_state', 'is_b2b', 'is_pitaya', 'diseases_sev', 'tds_cost',
       'internacao_proced_cost', 'exam_cost', 'consulta_cost', 'er_cost',
       'alice_therapy_cost', 'other_costs', 'total_care_cost'],
      dtype='object')

In [204]:
df['diseases_'] = df['diseases_sev'].str.split(';')
df2 = df.explode('diseases_')
df2['cid_'] = df2['diseases_'].apply(lambda x: str(x).strip()[0:3])
df3 = df2[['member_internal_code','calendar_date', 'cid_']].drop_duplicates()

In [205]:
df3

,member_internal_code,calendar_date,cid_
0,NC1000Z,2022-05-31,nan
1,NC1000Z,2022-06-30,nan
2,NC1000Z,2022-07-31,nan
3,NC1002P,2021-12-31,nan
4,NC1002P,2022-01-31,nan
...,...,...,...
114409,NC1ZZWN,2022-05-31,R74
114410,NC1ZZWN,2022-06-30,R74
114411,NC1ZZWN,2022-07-31,R74
114412,NC1ZZWN,2022-08-31,N17


In [206]:
df_row = pd.get_dummies(df3['cid_'], prefix = 'cid')
df_row_2 = df3[['member_internal_code', 'calendar_date']].merge(df_row, how = 'inner', left_index=True, right_index=True)
df_row_3 = df_row_2.groupby(['member_internal_code', 'calendar_date']).max().reset_index()
df_row_3.sort_values(by=['member_internal_code', 'calendar_date'], inplace=True)
#create a cid filter
df_row_3.drop(columns = 'cid_nan', inplace = True)
cids_cols = [col for col in df_row_3 if col.startswith('cid_')]
df_row_3[cids_cols] = df_row_3[cids_cols].apply(pd.to_numeric).fillna(0)

***

In [207]:
df_filled = df.merge(
                    pd.get_dummies(df.current_state, prefix = 'uf'),how = 'inner', left_index=True, right_index=True).merge(
                        pd.get_dummies(df.current_city, prefix = 'city'),how = 'inner', left_index=True, right_index=True).drop(columns=['current_state', 'current_city'])
df_filled['is_male'] = np.where(df_filled['current_sex']=='MALE',1,0)
df_filled['is_trans'] = np.where(df_filled['current_sex']!=df_filled['current_gender'],1,0)
#create the base-dummies
df_filled.drop(columns=['current_gender','current_sex','city_São Paulo', 'uf_SP',
                        'diseases_sev','tds_cost', 'exam_cost', 'consulta_cost',
                        'internacao_proced_cost', 'er_cost',
                        'alice_therapy_cost', 'other_costs'], inplace=True)
df_filled.drop(columns=['diseases_'], inplace=True)

base_covariates_list = df_filled.drop(columns = ['member_internal_code', 'calendar_date', 'mth_order','total_n_mth', 'total_care_cost']).columns.to_list()
core_covariates = ['member_age', 'is_pitaya', 'is_male']
y = ['total_care_cost']

### Merge bases

In [208]:
ml_df = df_filled.merge(df_row_3, how = 'inner', left_on=['member_internal_code', 'calendar_date'], right_on = ['member_internal_code', 'calendar_date'])

#adjust variables
ml_df['n_total_cids'] = ml_df[cids_cols].sum(axis=1)
ml_df['ref_mth'] = (pd.to_datetime(ml_df['calendar_date']).dt.to_period('M') -   pd.to_datetime(['1970-01-31']).to_period('M')).apply(lambda x: x.n)

#entry-calendar date
ml_df['v1'] = np.where(ml_df['mth_order']==1, ml_df['ref_mth'], np.nan)
ml_df = ml_df.merge(ml_df.groupby(['member_internal_code'])['v1'].min().rename('ref_mth_entrance_group'), how = 'inner', left_on = 'member_internal_code', right_index = True)
ml_df['ref_mth_entrance_group'] = ml_df['ref_mth_entrance_group'].astype('Int64')
ml_df.drop(columns = ['v1'], inplace = True)


In [209]:
ml_df_calendar_base = ml_df[['member_internal_code','calendar_date','mth_order','ref_mth', 'ref_mth_entrance_group']]

***

In [210]:
accountable_cids = ml_df.groupby('member_internal_code')[cids_cols].min().drop_duplicates().sum(axis=0).sort_values(ascending=False)
accountable_cids = accountable_cids[accountable_cids>=10].index.to_list()

In [211]:
entry_list_cids = []
diagnosed_list  = []

for cid in accountable_cids:
    print('current cid:', cid)
    #cid_list not including the cid
    cid_list = [c for c in cids_cols if c!=cid]

    temp_df = pd.concat([ml_df_calendar_base, ml_df[cid]], axis = 1)
    temp_df.sort_values(by = ['member_internal_code', 'calendar_date'], inplace = True)
    #
    lag_name = cid + '_lag_1'
    temp_df = pd.concat([temp_df,temp_df.groupby(['member_internal_code'])[cid].shift(1).rename(lag_name)], axis = 1)
    temp_df.sort_values(by = ['member_internal_code', 'calendar_date'], inplace = True)
    #
    entry_name = 'entry_' + cid
    diag_name = 'diag_' + cid
    #
    temp_df['v1'] = np.where((temp_df[cid]==1) & (temp_df['mth_order']==1), 1, 0)
    temp_df['ind_diag_mth'] = np.where((temp_df[cid]==1) & (temp_df[lag_name]!=1) & (temp_df['mth_order']>=2), 1, 0)
    temp_df = temp_df.merge(temp_df.groupby(['member_internal_code'])['v1'].max().rename(entry_name), how = 'inner', left_on = 'member_internal_code', right_index = True)
    temp_df = temp_df.merge(temp_df.groupby(['member_internal_code'])['ind_diag_mth'].max().rename(diag_name), how = 'inner', left_on = 'member_internal_code', right_index = True)
    temp_df.drop(columns = ['v1',lag_name], inplace = True)

    if temp_df[entry_name].mean()>0:
        
        #DF to estimate variables
        #treat_group_mths = temp_df[temp_df[diag_name]==1]['ref_mth_entrance_group'].unique()
        #control_group = temp_df[(temp_df['ref_mth_entrance_group'].isin(treat_group_mths)) & (temp_df[diag_name]==0) & (temp_df[diag_name]==0)]
        #total_df = pd.concat([temp_df[temp_df[diag_name]==1], control_group], axis = 0)
        #df = ml_df[[base_covariates_list + cid_list + y + ['mth_order', 'ref_mth_entrance_group']].merge(total_df[[diag_name] + ['diag_mth']], how = 'inner', left_index = True, right_index = True)

        entry_list_cids.append(entry_name)

    if temp_df[diag_name].mean()>0:
        diagnosed_list.append(diag_name)

 
    

current cid: cid_H52
current cid: cid_F41
current cid: cid_E66
current cid: cid_F32
current cid: cid_J30
current cid: cid_K29
current cid: cid_Z72
current cid: cid_J34
current cid: cid_G47
current cid: cid_G43
current cid: cid_K21
current cid: cid_J45
current cid: cid_P01
current cid: cid_E28
current cid: cid_E03
current cid: cid_N20
current cid: cid_M54
current cid: cid_I84
current cid: cid_M23
current cid: cid_L70
current cid: cid_F33
current cid: cid_J35
current cid: cid_R74
current cid: cid_Z44
current cid: cid_N63
current cid: cid_P74
current cid: cid_E78
current cid: cid_M41
current cid: cid_J32
current cid: cid_B97
current cid: cid_S93
current cid: cid_I10
current cid: cid_J31
current cid: cid_F90
current cid: cid_T83
current cid: cid_U07
current cid: cid_N47
current cid: cid_N80
current cid: cid_Z32
current cid: cid_K30
current cid: cid_E07
current cid: cid_I83
current cid: cid_N94
current cid: cid_Z97
current cid: cid_F51
current cid: cid_M51
current cid: cid_F50
current cid: 

#### Test for  entry

In [212]:
treat_group_mths = temp_df[temp_df[diag_name]==1]['ref_mth_entrance_group'].unique()
control_group = temp_df[(temp_df['ref_mth_entrance_group'].isin(treat_group_mths)) & (temp_df[diag_name]==0) & (temp_df[diag_name]==0)]
total_df = pd.concat([temp_df[temp_df[diag_name]==1], control_group], axis = 0)
df = ml_df[base_covariates_list + cid_list + y + ['member_internal_code','mth_order', 'ref_mth_entrance_group']].merge(total_df[[diag_name] + ['ind_diag_mth']], how = 'inner', left_index = True, right_index = True)
df = df.merge(df[df['ind_diag_mth']==1].groupby('member_internal_code')['mth_order'].max().rename('diag_mth_order'), how = 'left', left_on = 'member_internal_code', right_index = True)

In [213]:
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

logit = LogisticRegression()

# define pipeline
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

In [214]:
X = df[df['mth_order']==1][base_covariates_list + cid_list]
y = df[df['mth_order']==1]['diag_cid_X06']

# transform the dataset
X, y = pipeline.fit_resample(X, y)

clf = logit.fit(X,y)

pre_treat = clf.predict(df[df['mth_order']==1][base_covariates_list + cid_list])
df.loc[df['mth_order']==1, 'pre_treat'] = pre_treat

In [215]:
df = df.merge(df.groupby(['member_internal_code'])['pre_treat'].max().rename('pre_treat_max'), how = 'inner', left_on = 'member_internal_code', right_index = True)

In [216]:
df['treat'] = np.where(df['diag_cid_X06']==1, 1, 0)
df.loc[(df['treat']==0) & (df['pre_treat_max']==0), 'treat'] = np.nan

month_list = df[df['diag_cid_X06']==1]['ref_mth_entrance_group'].unique()
ml_df = df[df['ref_mth_entrance_group'].isin(month_list) & (df['treat'].notna())][['member_internal_code','mth_order','ref_mth_entrance_group','ind_diag_mth','diag_mth_order','treat', 'total_care_cost'] + core_covariates]

***

In [217]:
ml_df

,member_internal_code,mth_order,ref_mth_entrance_group,ind_diag_mth,diag_mth_order,treat,total_care_cost,member_age,is_pitaya,is_male
0,NC1000Z,1,628,0,NaN,0.0,0.000000,33.0,0,0
1,NC1000Z,2,628,0,NaN,0.0,0.000000,33.0,0,0
2,NC1000Z,3,628,0,NaN,0.0,16.019120,34.0,0,0
98,NC1015C,1,615,0,NaN,0.0,0.000000,31.0,0,0
99,NC1015C,2,615,0,NaN,0.0,755.871193,31.0,0,0
...,...,...,...,...,...,...,...,...,...,...
114310,NC1ZYWB,6,624,0,NaN,0.0,1954.909917,37.0,0,0
114311,NC1ZYWB,7,624,0,NaN,0.0,1635.984687,37.0,0,0
114312,NC1ZYWB,8,624,0,NaN,0.0,1021.145917,37.0,0,0
114313,NC1ZYY3,1,626,0,NaN,0.0,982.902339,25.0,0,0


In [179]:
df = pd.DataFrame()
for pr_mth in ml_df['ref_mth_entrance_group'].unique():
    diag_months = ml_df[(ml_df['ind_diag_mth']==1) & (ml_df['ref_mth_entrance_group']==pr_mth)]['diag_mth_order'].unique()
    print(pr_mth, diag_months)
    for mth in diag_months:
        treat_df = ml_df[(ml_df['ref_mth_entrance_group']==pr_mth) & (ml_df['treat']==1) & (ml_df['diag_mth_order']==mth)]
        treat_df['ev_an_mth'] = treat_df['mth_order'] - mth
        control_df = ml_df[(ml_df['ref_mth_entrance_group']==pr_mth) & (ml_df['treat']==0)]
        control_df['ev_an_mth'] = control_df['mth_order'] - mth

        df = pd.concat([df,treat_df, control_df], axis = 0)


628 [4. 2. 3.]
615 [13.]
617 [10.  4.]
626 [3.]
624 [5.]
620 [ 3. 10.]
622 [2.]
625 [4.]
621 [2.]
627 [4.]
623 [5. 6.]


In [194]:
final_ml_df = df.sort_values(['member_internal_code','ev_an_mth'])
final_ml_df.drop(columns = ['mth_order', 'diag_mth_order', 'ind_diag_mth'], inplace = True)

In [199]:
final_ml_df.groupby(['treat', 'ev_an_mth'])['total_care_cost'].mean()

treat  ev_an_mth
0.0    -12.0         282.837843
       -11.0         844.055049
       -10.0         829.134406
       -9.0          553.689201
       -8.0         1144.143509
       -7.0         1044.792041
       -6.0         1099.430356
       -5.0          724.894403
       -4.0          659.180201
       -3.0          650.978967
       -2.0          758.638658
       -1.0          852.490920
        0.0          889.349156
        1.0          894.672511
        2.0          855.421042
        3.0          895.535932
        4.0          966.449751
        5.0          977.956478
        6.0         1134.735018
        7.0          927.112239
        8.0          924.210522
        9.0          887.225929
        10.0        1278.521167
        11.0        1344.892111
1.0    -12.0         420.417922
       -11.0         688.006881
       -10.0        1525.963141
       -9.0          350.125671
       -8.0         1766.932417
       -7.0         1178.170393
       -6.0         112

In [ ]:
ml_df.to_csv('src/data/panel_cc_cids_did_df.csv', index = False)